In [1]:
%pip install openai
%pip install azure-core
%pip install pandas
%pip install chromadb

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 10, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 136.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-27dcd60a-7331-427c-b45e-eb6c42209168
    Can't uninstall 'typing_extensions'. No files were found to uninst

In [2]:
import os
import json
from openai import AzureOpenAI
import pandas as pd
import numpy as np
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 12, Finished, Available, Finished)

#### load reviews csv

In [3]:

pd.set_option('display.max_colwidth', None)

df = pd.read_csv('/lakehouse/default/Files/reviews.csv', nrows=200)
df= df.dropna()
df.head()


StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 13, Finished, Available, Finished)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,3/30/2009,10952,Lam,Daniel is really cool. The place was nice and clean. Very quiet neighborhood. He had maps and a lonely planet guide book in the room for you to use. I didnt have any trouble finding the place from Central Station. I would defintely come back! Thanks!
1,515749,1671407,7/9/2012,2640670,Gregory,"If you want the authentic Amsterdam houseboat experience, this is it! It is a great boat, located on a quiet canal with a view of the Rijksmuseum towers. Spotlessly clean, comfortable bed, hot shower, good wifi signal, nice kitchen with fridge, cofee maker, and stove for cooking meals. The location is very accessible to museums, gym bar and bakery on the corner, nine streets area with lots of resaraunts and bars and jazz clubs, Vondelpark, shopping on Utrechtstraat, and all of the sites of central Amsterdam. \r<br/>\r<br/>Now for the special part: If you are at all interested in boats and maritime history, this one is a great find. The cabin where guests stay is actually the cargo hold within the original hull of a freight boat that once hauled goods all over Holland. (Just imagine the countless tons of cargo that came in and out of the place where you are sleeping!) And yet the interior beautifully and comfortably remodeled using new and salvaged materials including a number of hatches that serve as skylights. The original steerhouse is still intact, along with the original one-cylinder engine down below. But the highlight is the captain's quarters where the family who owned and operated the boat lived during the 1920s (not open for guests to sleep in, yet). Stepping down into the quarters, complete with original cabinets and cookstove, feels like traveling back in time. The host cares a lot about preserving the historical character of his boat, because most other boats like it have long since gone to the scrapyard. He showed us fascinating photographs of the original owners who lived on the boat when it plied the canals nearly 100 years ago. \r<br/>\r<br/>There are many houseboats in Amsterdam. A lot of them are glorified trailer houses on flat barges. This one is a gem."
2,515749,1715674,7/15/2012,1032804,Michael,"Unique and luxurious to be sure. I couldn't recommend staying with Derk any more highly.\r<br/>\r<br/>Coolest thing first: We showed up to Derk's and he actually had three houseboats in a row, and offered to show up the one we saw on the site and another, and let us choose our favorite to stay in! We had an incredible boat with wired internet, a comfortable bed and a very comfortable futon, TV, speakers, hot water, a shower, washer/dryer. I couldn't believe I was on a boat. Plus the charm of being on one was fantastic, and the views out onto the canal were fantastic. It rained one morning and was such a beautiful way to wake up.\r<br/>\r<br/>In addition, Derk was really communicative, responded to our last minute (day before!) request for a reservation quite casually, and met us right at the boat. He even had a few cold beers in the fridge for us, and was really great explaining the area.\r<br/>\r<br/>Would recommend!"
3,2818,1771,4/24/2009,12798,Alice,"Daniel is the most amazing host! His place is extremely clean, and he provides everything you could possibly want (comfy bed, guidebooks & maps, mini-fridge, towels, even toiletries). He is extremely friendly and helpful, and will go out of his way to help you if needed, or just let you come and go as you please. Highly recommended!"
4,515749,1963378,8/12/2012,503786,Brian,"My wife and I recently stopped in Amsterdam for a few days during our honeymoon. We didn't want to stay in the typical Amsterdam hotel and opted for something special - a traditional Dutch houseboat. Needless to say staying on a boat was a special experience, but the boat was very clean, was on a quiet canal in a superb location in central Amsterdam, and had everything we needed for a memorable weekend. The owner was good

#### get embedding using OpenAI SDK

In [ ]:
endpoint = "https://aihub-openai-base-model.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"
model_name = "text-embedding-ada-002"
deployment = "gpt-4.1"

subscription_key = ""
api_version = "2024-12-01-preview"
response_format={"type": "json_object"},

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

# get embedding by text
def get_embedding(text):
    text = text.replace('\n', text)
    text = text.replace('<br />', text)

    response = client.embeddings.create(input=text, model='text-embedding-ada-00')

    return response.data[0].embedding


def cosine_similarity(a_embedding, b_embedding):
    cs = np.dot(a_embedding, b_embedding) / (np.linalg.norm(a_embedding) * np.linalg.norm(b_embedding)) #np.linalg.norm(a_embedding - b_embedding)
    return cs


def search_reviews(search_text):
    search_text_embedding = get_embedding(search_text)
    df['similarities'] = df.embeddings.apply(lambda x: cosine_similarity(x, search_text_embedding))
    result = df.sort_values('similarities', ascending=False).head(10)
    return result



StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 14, Finished, Available, Finished)

In [5]:
df['embeddings'] = df.comments.apply(lambda x: get_embedding)

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 15, Finished, Available, Finished)

In [6]:
embedding_output_path = '/lakehouse/default/Files/airbnb_reviews_embedding.csv'

df.to_csv(embedding_output_path)

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 16, Finished, Available, Finished)

#### get embedding using chromaDB SDK

In [7]:
openai_ef = embedding_functions.(
    api_key= subscription_key,
    model_name = model_name,

)

chromadb_client = chromadb.Client()

collection = chromadb_client.create_collection(name="reviews", embedding_function=openai_ef)

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 17, Finished, Available, Finished)

In [8]:
comments = df['comments'].tolist()
ids = df['id'].map(lambda x: str(x)).tolist()

collection.add(
    documents=comments,
    ids=ids
)


StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, 18, Finished, Available, Finished)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: CkyMEjIl************************************************************************PVdc. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}} in add.

In [ ]:
collection.query(
    query_text=['walkable'],
    n_results=5
)

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, -1, Cancelled, , Cancelled)

#### search with cosine similarity score

In [ ]:
search_text = 'coffee'

result = df.sort_values('similarities', ascending=False).head(10)

search_text_embedding = get_embedding(search_text)

# calculate cosine similarity
df['similarities'] = df.embeddings.apply(lambda x: cosine_similarity(x, search_text_embedding))

result = df.sort_values('similarities', ascending=False).head(10)

result = result.drop(columns=[x for x in result.columns if x not in ['comments','similarities']])

result

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, -1, Cancelled, , Cancelled)

In [ ]:


a = df.embeddings[0]
b = get_embedding('comfortable')
c = cosine_similarity(a,b)

c

StatementMeta(, ffa3f3fc-1fe1-4e68-b01a-8c403f68351f, -1, Cancelled, , Cancelled)